<a href="https://colab.research.google.com/github/nagamamo/financial_data_science/blob/main/6_2_option_pricing_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

6.2　オプション価格理論の実装

　本例では、オプション価格を算出するためのパッケージであるoption-price 0.2.0を用いてコールオプション価格の算出事例を示します。この算出に際し、Google Collaboratoryでは、option-priceのインストールは、個々のコードセルにおいて実行します。

In [ ]:
pip install option-price

　パッケージのインストール後、3つのライブラリをインポートします。ひとつはoption-priceからオプション価格算出のためのライブラリOptionです。他の2つは数値計算用のpandasとnumpyです。

In [2]:
#[1]ライブラリの読み込み
from optionprice import Option#オプション価格算出用ライブラリ
import pandas as pd
import numpy as np

　次に株価と金利の日次データを取得します。日経平均株価は2018年初から2022年末までの日次データをURLから取得します。このデータをpandasデータフレームnikkeiに保存し、numpyにより対数値の差分を計算してリターンを算出、そのデータフレーム名をd_nikkeiとします。加えて、オプション価格の算出に必要な日経平均株価と１カ月物金利の直近の値を、それぞれlast_nikkei、last_rateというデータフレーム名で保存します。

In [3]:
#[2]データ読込み
url = "https://github.com/nagamamo/data/blob/main/5_nikkei225_data.csv?raw=true"#Git-hubからCSVデータの入手
nikkei = pd.read_csv(url)#データフレームの作成
nikkei = nikkei.set_index("Date")#日時をインデックスへ変更

In [4]:
#[3]データ前処理
d_nikkei=np.log(nikkei.nikkei).diff(1)
last_nikkei=nikkei.nikkei[-1]
last_rate=0.01

　続いて、オプション価格を算出するための初期値を設定します。株価ｓ0は直近値であるlast_nikkeiを指定し、オプションの購入者が判断する行使価格は、ここでは日経平均株価29,000円と設定します。行使期間を30日間とし、これに合わせ無リスク資産金利は1カ月物金利last_rateを外生的に与えます 。最後に日経平均株価変動率d_nikkeiから標準偏差を算出し、この値をsigmaとします。

In [5]:
#[4]初期値の設定
s0=last_nikkei#株価直近値を指定
k=29000#行使価格
r=last_rate/100#円金利直近値を指定
days=30#行使期間
sigma=np.std(d_nikkei)#株価標準偏差を算出

In [ ]:
#[5]プット・オプション価格の算出
#some_option = Option(european=True,kind='put',s0=s0,k=k,t=days,sigma=sigma,r=r,dv=0)#日数により期間を指定するケース                    

 最後にOptionを用い、コール・オプション価格を算出します。以下のソースコードは、データフレーム名がsome_optionであり、kind='call'によりコール・オプション価格であることを指定しています。原資産価格（株価）の直近値s0、行使価格k、（予想）ボラティリティsigma、無リスク資産円金利1カ月r、行使期間daysと指定します。このケースでは、配当率はdv=0の「なし」としました。ヨーロピアン・オプション価格を算出する場合であれば、european=Trueとし、アメリカン・オプションの場合にはeuropean=Falseとします。

In [6]:
#[5]コール・オプション価格の算出
some_option = Option(european=True,kind='call',s0=s0,k=k,sigma=sigma,r=r,t=days,dv=0)#日数により期間を指定するケース
#some_option = Option(european=True,kind='call',s0=s0,k=k,sigma=sigma,r=r,start='2022-07-24',end='2022-08-23',dv=0)#日付により期間を指定するケース

  算出結果は、Google Colabの場合、データフレーム名を記述すれば表示されます。本例では、オプション価格は1.241％と算出されています。

In [7]:
#[6]算出結果の表示
print(some_option)

Type:           European
Kind:           call
Price initial:  27968.99
Price strike:   29000
Volatility:     1.2409896519934491%
Risk free rate: 0.01%
Start Date:     2023-01-04
Expire Date:    2023-02-03
Time span:      30 days
